# Get Available Art Movements in Wikidata

In [ ]:
from utils.wikidata_querier import get_all_instaces_of_category
import pandas as pd

category_id = "wd:Q968159" # Art movement
movements = get_all_instaces_of_category(category_id)
df_mov = pd.DataFrame(movements)
print(f"Found {len(movements)} movements!")
print(df_mov.head(5))

# Get All People involved with the Art Nouveau Movement

In [ ]:
from utils.wikidata_querier import get_wiki_persons_from_movement
movement_label = "Art Nouveau"
movement_id="wd:Q34636"
people_all, people_essential = get_wiki_persons_from_movement(movement_id)
df_per = pd.DataFrame(people_essential)
print(f"Found {len(people_essential)} people involved with {movement_label}!")
print(df_per.head())

# Get the Wikipedia Biographies of each participant involved in the movement

In [ ]:
import os, time
from utils.utils_wiki import get_wikipedia_article, save_wikipedia_page

output_dir = f"data/wikipedia/{movement_label.replace(' ','_')}"
if not os.path.exists(output_dir): os.mkdir(output_dir)

stop = 0
for wiki_url in df_per["wiki_page"]:
    page = get_wikipedia_article(query_url=wiki_url)
    if page:
        save_wikipedia_page(page, output_path=f"{output_dir}/{page.title.replace(' ', '_').lower()}.txt", include_metadata=True, include_sections=True, include_infobox=True)
        time.sleep(5)
        stop += 1
    if stop == 5: break

In [ ]:
! python en_text_to_json_flair.py 'data/wikipedia/Art_Nouveau'